In [3]:
import duckdb

sql='''
SELECT
    LOWER(abc.TEST_NAME) AS test_name,
    abc.TEST_VERSION,
    abc.TEST_TYPE AS test_type,
    abc.VERSION_START_DATE AS test_start_date,
    CASE
        WHEN abc.VERSION_END_DATE IS NULL THEN '2024-01-01'
        ELSE abc.VERSION_END_DATE
    END AS test_end_date,
    abc.STATUS AS TEST_STATUS,
    abc.TEST_METRICS
FROM history.ab_test_dimension abc -- Use the source defined in sources.yml
    LEFT JOIN abtest.ab_test_dashboard abtd
    ON LOWER(abc.TEST_NAME) = abtd.TEST_NAME
    AND abc.TEST_VERSION = abtd.TEST_VERSION
WHERE (
        abc.STATUS = 'LIVE'
        OR (
            abc.STATUS = 'ENDED'
            AND (
                abtd.test_status = 'LIVE'
                OR abtd.test_status IS NULL
            )
        )
    ) -- include live tests or tests that have just ended today
    AND NOT UPPER(abc.TEST_NAME) LIKE '%AA%' -- exclude AA tests
    AND NOT abc.VARIATION LIKE '%100.00%' -- exclude tests that have been released (aka, variant ramped to 100%)
    AND abc.TEST_METRICS IS NOT NULL -- exclude tests that don't have any metrics
    AND abc.VERSION_END_DATE > '2024-01-01' -- exclude tests that ended over 2 weeks ago (for initial run)
    AND (
        abtd.END_DATE != '2024-01-01'
        OR abtd.END_DATE IS NULL
    ) -- exclude tests that have been processed today already
    AND LOWER(abc.TEST_NAME) not in (
        'proactive-order-monitoring-10kto15k'
    ) -- exclude anchor list tests (need to find a better way to do this)
ORDER BY test_start_date DESC -- start with the newest tests
;

'''

with duckdb.connect('dbt_analytics/data/local.duckdb') as conn:
    display(conn.sql(sql))


┌───────────┬──────────────┬───────────┬─────────────────┬───────────────┬─────────────┬──────────────┐
│ test_name │ TEST_VERSION │ test_type │ test_start_date │ test_end_date │ TEST_STATUS │ TEST_METRICS │
│  varchar  │    int64     │  varchar  │      date       │     date      │   varchar   │  varchar[]   │
├───────────┴──────────────┴───────────┴─────────────────┴───────────────┴─────────────┴──────────────┤
│                                               0 rows                                                │
└─────────────────────────────────────────────────────────────────────────────────────────────────────┘